# Exploring Ocean Color with NASA PACE (Teen Volunteer Session)

Today you’ll:
- Look at chlorophyll in the Pacific Northwest
- Make a real satellite map using Python
- Change the map to explore other regions

This is NASA science data — you’re just using it in a simpler way.

In [ ]:
# Imports
import earthaccess
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## Login to NASA Earthdata

If this is your first time using NASA Earthdata, you may be asked to log in.

In [ ]:
earthaccess.login(persist=True)

## Load a PACE monthly Level-3 chlorophyll file

We’ll use a monthly, gridded product (Level-3) so it’s quick to plot.

Tip: If you want a different month later, change the `temporal` dates.

In [ ]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL",
    temporal=("2024-06-01", "2024-06-30"),
    granule_name="*.MO.*.0p1deg.*"
)

files = earthaccess.open(results)
ds = xr.open_dataset(files[0])
ds

## Choose a region (Pacific Northwest)

These numbers define a rectangle:
- **Latitude:** 40 to 52
- **Longitude:** -130 to -120

Try changing them to explore other places (e.g., Puget Sound vs. Oregon coast).

In [ ]:
lat_min, lat_max = 40, 52
lon_min, lon_max = -130, -120

chl = ds["CHL"].sel(
    lat=slice(lat_max, lat_min),
    lon=slice(lon_min, lon_max)
)
chl

## Make a map

In [ ]:
fig = plt.figure(figsize=(7, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

chl.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    robust=True,
    cmap="viridis",
    cbar_kwargs={"label": "Chlorophyll (mg m⁻³)"}
)

ax.coastlines()
ax.add_feature(cfeature.LAND, facecolor="lightgray")
ax.set_extent([lon_min, lon_max, lat_min, lat_max])

ax.set_title("PACE Monthly Chlorophyll – Pacific Northwest")
plt.show()

## Try this!

Change the `lat_min`, `lat_max`, `lon_min`, `lon_max` values above and re-run.

Ideas:
- Zoom into **Puget Sound** (smaller box around Seattle)
- Move south toward **Oregon**
- Compare **coast vs open ocean**
- Zoom way out and see what patterns appear

## Other PACE Level-3 products to explore

If you want to try other images, swap the `short_name` in the search cell.

- `PACE_OCI_L3M_SST` — Sea surface temperature
- `PACE_OCI_L3M_KD` — Water clarity (diffuse attenuation)
- `PACE_OCI_L3M_RRS` — Ocean reflectance (light itself)
- `PACE_OCI_L3M_POC` — Particulate organic carbon

Not every product will be available for every date — that’s normal.